In [ ]:
import pandas as pd 
df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.drop(['id'],axis='columns',inplace=True)
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)
df.isnull().sum()

In [ ]:
df.groupby('stroke')['heart_disease'].count()

In [ ]:
count_0,count_1 = df.stroke.value_counts()
count_0,count_1 

In [ ]:
df1_class_1 = df[df.stroke == 1]
df1_class_0 = df[df.stroke == 0].sample(count_1)

In [ ]:
df1 = pd.concat([df1_class_1,df1_class_0],axis=0)
df1.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist([df[df.stroke == 1].smoking_status,df[df.stroke == 0].smoking_status],label=['Stroke','has no Stroke'])
plt.legend()

In [ ]:
plt.hist([df1[df1.stroke == 1].smoking_status,df1[df1.stroke == 0].smoking_status],label=['Stroke','has no Stroke'])
plt.legend()

In [ ]:
plt.hist([df1[df1.stroke == 1].gender,df1[df1.stroke == 0].gender],label=['Stroke','has no Stroke'])
plt.legend()

In [ ]:
df1.ever_married.unique()

In [ ]:
df1.ever_married = df1.ever_married.apply(lambda x : 1 if x=='Yes' else 0)

In [ ]:
df1.ever_married.unique()

In [ ]:
jobs = df1.work_type.unique()

In [ ]:
dummies = pd.get_dummies(df1.work_type,drop_first=True)

In [ ]:
df1 = pd.concat([df1,dummies],axis=1)

In [ ]:
df1.Residence_type.unique()

In [ ]:
df1.Residence_type = df1.Residence_type.apply(lambda x : 1 if x=='Urban' else 0)

In [ ]:
df1.head()

In [ ]:
df.gender.unique()

In [ ]:
df.gender.value_counts()

In [ ]:
df1.gender = df1.gender.apply(lambda x: 0 if ((x == 'Female') or (x == 'Other')) else 1)

In [ ]:
df1.smoking_status.unique()

In [ ]:
df1.smoking_status.value_counts()

In [ ]:
dummies = pd.get_dummies(df1.smoking_status,drop_first=True)
df1 = pd.concat([df1,dummies],axis=1)

In [ ]:
df1.shape

In [ ]:
df1.head()

In [ ]:
df1.drop(['work_type','smoking_status'],axis='columns',inplace=True)

In [ ]:
df1.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1.drop(['stroke'],axis='columns'),df1.stroke,test_size=0.2,stratify=df1.stroke)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

# method 2: over sampling

In [ ]:
count_0 , count_1

In [ ]:
df2_count_1 = df[df.stroke==1].sample(count_0,replace=True)
df2_count_0 = df[df.stroke==0]

In [ ]:
df2 = pd.concat([df2_count_1,df2_count_0],axis=0)
df2.shape

In [ ]:
def covert(df1):
    df1.ever_married = df1.ever_married.apply(lambda x : 1 if x=='Yes' else 0)
    dummies = pd.get_dummies(df1.work_type,drop_first=True)
    df1 = pd.concat([df1,dummies],axis=1)
    df1.Residence_type = df1.Residence_type.apply(lambda x : 1 if x=='Urban' else 0)
    df1.gender = df1.gender.apply(lambda x: 0 if ((x == 'Female') or (x == 'Other')) else 1)
    dummies = pd.get_dummies(df1.smoking_status,drop_first=True)
    df1 = pd.concat([df1,dummies],axis=1)
    df1.drop(['work_type','smoking_status'],axis='columns',inplace=True)
    return df1

In [ ]:
df2.dtypes

In [ ]:
df2 = covert(df2)
df2.dtypes

In [ ]:
df2.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df2.drop(['stroke'],axis='columns'),df2.stroke,test_size=0.3,stratify=df2.stroke)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
import pickle
with open('model_pickle','wb') as file:
    pickle.dump(model,file)

In [ ]:
len(X_test.columns)

In [ ]:
X_test.columns

In [ ]:
work = ['Never_worked','Private','Self-employed','Self-employed']
def getindex(array,string):
    for i in range(len(array)):
        if array[i] == string:
            return i
    return None
getindex(work,'Private')

In [ ]:
def predict(gender,age,hypertension,heart_disease,ever_married,Residence_type,avg_glucose_level,bmi,work_type,smoking_status):
    import numpy as np
    work = ['Never_worked','Private','Self-employed','Self-employed']
    smoking = ['formerly smoked', 'never smoked','smokes']
    with open('model_pickle','rb') as file:
        mp = pickle.load(file)
    x = np.zeros(15)
    if gender == 'male': x[0]=1 
    else:x[0]=0
    x[1]=age
    x[2] = hypertension
    x[3] = heart_disease
    if ever_married == 'Yes':x[4]=1 
    else:x[4]=0
    if Residence_type == 'Urban':x[5]=1 
    else:x[5]=0
    x[6] = avg_glucose_level
    x[7] = bmi
    work_index =getindex(work,work_type)
    smok_index = getindex(smoking,smoking_status)
    if(work_index):x[8+work_index] = 1
    if(smok_index):x[12+smok_index] = 1
    print(x)
    return mp.predict([x])

In [ ]:
predict('Female',61,0,1,'Yes','Rural',120.46,36.8,'Govt_job','smokes')

In [ ]:
predict('male',53,0,0,'Yes','Rural',120.46,36.8,'Private','formerly smoked')